# Исследование надежности заемщиков

- Заказчик — кредитный отдел банка. 
- Задача: разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
- Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd

Скачаем и рассмотрим данные.

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В данных присутствуют пропуски. Также есть аномалии, например, отрицательные значения в столбце days_employed и children

Пропуски в столбце с количеством дней, прошедших от устройства, можно объяснить разными причинами: человек никогда не работал, записи о его деятельности сохранились в бумажной трудовой и были неккоректно перенесены, их не указали. Так как в солбце общего дохода такое же количество пропусков, сколько и в столбце с днями, в тех же строках, можно предположить, что этот столбец рассчитывается из количества дней, поэтому при отсутствии данных в столбце с днями данные отсутсвуют и в нем. Также стоит учитывать технический фактор (например, ошибка при выгрузке).

## Предобработка данных

Оценим количество пропусков в данных 

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом. Пропуски в days_employed заполним немного позже.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Исправим аномалии в столбце days_employed. Заменим отрицательные значения на положительные

In [8]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Но этот столбец не понадобится нам для исследования, поэтому оставим как есть.

Рассмотрим столбец children

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце children есть два аномальных значения. Удалим строки, в которых они встречаются.

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Теперь аномальные значения отсутствуют

Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type.

Так как мы используем количественные данные для рассчета, вставлять на место пропусков, например категориальные данные не имеет смысла, так как это приведет к ошибкам в коде при рассчете. Если использовать категориальную замену в сочетании с try-except или вставить нули - это приведет к искажению данных, особенно, если пропусков относительно много. Среднее значение не учитывает выбросов данных, что может привести к ошибке. При этом вставка медианного значение учитывает выбросы и не искажает данные в такой степени, как нули или категориальные значения.

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Для удобства заменим вещественный тип данных в столбце total_income на int

In [15]:
data['total_income'] = data['total_income'].astype(int)

Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

In [16]:
data['education'] = data['education'].str.lower()

In [17]:
data.duplicated().sum()

71

Удалим дубликаты.

In [18]:
data = data.drop_duplicates()

Создадим категоризацию данных по доходу. Пусть будет присвоены следующие категории на основании дохода:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [19]:
#функция, реализующая категоризацию на основании приведенных выше диапазонов
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

Применим функцию

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Рассмотрим перечень уникальных целей взятия кредита.

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Для обобщения данных создадим категоризацию целей взятия кредита. Пусть будет разделение на следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [22]:
#функция, категоризирующая данные на основе имеющихся словесных вхождений
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

Применим функцию к данным

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

Проанализируем, есть ли зависимость между количеством детей и возвратом кредита в срок:

In [24]:
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


Явная зависимость наблюдается при рассмотрении до 2-х детей, процент должников увеличивается с каждым новым ребенком. При этом, так как большее количество детей - более редкий случай, выводы о плательщиках с 3-5 детьми делаются по слишком малой выборке, чтобы определить наличие или отсуствие зависимости. Но при таком наборе данных, в этом диапазоне зависимости нет.

Проанализируем, есть ли зависимость между семейным положением и возвратом кредита в срок

In [25]:
print(data.groupby(by='family_status')['children'].count())

(data.groupby(by='family_status')['children'].sum() / data.groupby(by='family_status')['children'].count()).sort_values() 

family_status
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
гражданский брак          4134
женат / замужем          12261
Name: children, dtype: int64


family_status
вдовец / вдова           0.146162
Не женат / не замужем    0.225680
в разводе                0.429773
гражданский брак         0.457426
женат / замужем          0.565941
Name: children, dtype: float64

Вывод: Среди одиноких людей гораздо меньше должников, чем среди тех, кто находится в отношениях или состоял в них. Меньше всего должников среди овдовевших.

Проанализируем, есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [26]:
print(data.groupby(by='total_income_category')['debt'].count())

(data.groupby(by='total_income_category')['debt'].sum() / data.groupby(by='total_income_category')['debt'].count()).sort_values()

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: debt, dtype: int64


total_income_category
D    0.060172
B    0.070602
A    0.080000
C    0.084982
E    0.090909
Name: debt, dtype: float64

Вывод: Наиболее часто должниками становятся люди с наимешьим доходом из имеющейся выборки, в срок долг отдают чаще всего люди с дохом от 30 до 50 тысяч. При этом в каждой из категорий больше половины клиеинтов не возвращают долг в срок. Если посмотреть на распределение количества людей в каждой группе, то видно, что в выборке присутствует относительно малое количество людей с минимальным доходом и максимальным доходом, что говорит о том, что сравнивать эти группы с остальными не очень корректно.

Рассмотрим, как разные цели кредита влияют на его возврат в срок:

In [27]:
print(data.groupby(by='purpose_category')['debt'].count())

(data.groupby(by='purpose_category')['debt'].sum() / data.groupby(by='purpose_category')['debt'].count()).sort_values()


purpose_category
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313
Name: debt, dtype: int64


purpose_category
операции с недвижимостью    0.072551
проведение свадьбы          0.079118
получение образования       0.092528
операции с автомобилем      0.093480
Name: debt, dtype: float64

Вывод: В каждой из категорий больше двух третей клиентов не возвращают долг в срок. Наибольшее количество должников относится к получающим образование и совершающим операции с автомобилем.

## Общий вывод

 - Исходные данные содержат пропуски, которые могут быть вызваны как техническими, так и естественными причинами. Также есть аномалии. 
 - В ходе работы данные были приведены в удобный для использования вид: пропуски заполнены, аномалии устанены, данные частично категоризированы
 - Проведено исследование и сделан слежующий вывод: независимо от цели, очень много клиентов не оплачивает кредит вовремя. При этом наибольшее количество должников наблюдается среди людей, имеющих или имевших отношения. Отсутствие детей положительно влияет на срок оплаты кредита, этот фактор может быть связан с фактором наличия отношений. Логично, что люди в отношениях чаще заводят детей, чем люди без. Уровень дохода, судя по выборке, напрямую не влияет на возвращение долга в срок, но видно, что люди с наименьшим уровнем дохода чаще не укладываются в срок погашения кредита.